In [0]:
dbutils.widgets.text("volume_path", "")

In [0]:
volume_path = dbutils.widgets.get("volume_path")
if volume_path == "":
  raise Exception("Please provide a volume path")

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, ArrayType, DateType
from faker import Faker
import random
from datetime import datetime
import uuid

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, DateType, IntegerType, DoubleType
from faker import Faker
import random
import uuid
from datetime import datetime

spark = SparkSession.builder.appName("VentasElectrodomesticos").getOrCreate()

# ==================================
# 🔹 Funciones auxiliares
# ==================================
def generar_correo(nombre):
    dominios_buenos = ["gmail.com", "hotmail.com", "outlook.com", "yahoo.com"]
    dominios_erroneos = ["gamil.com", "hotmial.com", "outlok.com", "yahho.com"]
    dominio = random.choice(dominios_buenos + dominios_erroneos)
    nombre_base = nombre.lower().replace(" ", ".")
    return f"{nombre_base}{random.randint(1,999)}@{dominio}"

def generar_telefonos():
    n = random.randint(0, 3)
    if n == 0:
        return []
    telefonos = []
    for _ in range(n):
        if random.random() < 0.2:  # 20% chance de valor nulo
            telefonos.append(None)
        else:
            telefono = "".join([str(random.randint(0,9)) for _ in range(10)])
            telefonos.append(telefono)
    return telefonos

# ==================================
# 🧍‍♂️ Función: generar_clientes()
# ==================================
def generar_clientes(fake, number_of_customers):
    clientes_data = []
    id = 0
    for _ in range(number_of_customers):
        id += 1
        nombre = fake.name()
        genero = random.choice(["Masculino", "Femenino"])
        fecha_nacimiento = fake.date_of_birth(minimum_age=18, maximum_age=80)
        correos = [generar_correo(nombre) for _ in range(random.randint(1, 3))][0]
        telefonos = generar_telefonos()
        
        clientes_data.append({
            "id": id,
            "nombre": nombre,
            "genero": genero,
            "fecha_nacimiento": fecha_nacimiento,
            "correos": correos,
            "telefonos": telefonos
        })

    clientes_schema = StructType([
        StructField("id", StringType(), False),
        StructField("nombre", StringType(), True),
        StructField("genero", StringType(), True),
        StructField("fecha_nacimiento", DateType(), True),
        StructField("correos", StringType(), True),
        StructField("telefonos", ArrayType(StringType()), True)
    ])

    clientes_df = spark.createDataFrame(clientes_data, schema=clientes_schema)
    return clientes_df

# ==================================
# 📦 Función: generar_productos()
# ==================================
def generar_productos():
    productos = [
        ("Televisor", "Electrónica"),
        ("Refrigerador", "Cocina"),
        ("Lavadora", "Limpieza"),
        ("Microondas", "Cocina"),
        ("Aspiradora", "Limpieza"),
        ("Horno", "Cocina"),
        ("Licuadora", "Cocina"),
        ("Cafetera", "Cocina"),
        ("Plancha", "Limpieza"),
        ("Ventilador", "Climatización"),
        ("Secadora", "Limpieza"),
        ("Tostadora", "Cocina"),
        ("Batidora", "Cocina"),
        ("Parlante Bluetooth", "Audio"),
        ("Purificador de aire", "Climatización")
    ]

    productos_data = []
    for i, (producto, categoria) in enumerate(productos, start=1):
        productos_data.append({
            "id_product": i,
            "nombre_producto": producto,
            "precio_unitario": round(random.uniform(100, 2000), 2),
            "categoria": categoria
        })
    
    productos_schema = StructType([
        StructField("id_product", IntegerType(), False),
        StructField("nombre_producto", StringType(), True),
        StructField("precio_unitario", DoubleType(), True),
        StructField("categoria", StringType(), True)
    ])
    productos_df = spark.createDataFrame(productos_data, schema=productos_schema)
    return productos_df

# ==================================
# 💰 Función: generar_ventas()
# ==================================
def generar_ventas(fake, clientes_df, productos_df, num_ventas):
    clientes = [row.id for row in clientes_df.collect()]
    productos = productos_df.collect()
    
    ventas_data = []
    for _ in range(num_ventas):
        producto = random.choice(productos)
        customer_id = random.choice(clientes)
        unidades = random.randint(1, 5)
        descuento = round(random.uniform(0, 0.25), 2)  # entre 0% y 25%
        precio_unitario = producto["precio_unitario"]

        ventas_data.append({
            "id_order": str(uuid.uuid4()),
            "customer_id": customer_id,
            "id_product": producto["id_product"],
            "valor_unitario": precio_unitario,
            "valor_descuento": descuento,
            "unidades": unidades,
            "fecha_venta": fake.date_between(start_date='-1y', end_date='today')
        })
    
    ventas_schema = StructType([
        StructField("id_order", StringType(), False),
        StructField("customer_id", StringType(), True),
        StructField("id_product", IntegerType(), True),
        StructField("valor_unitario", DoubleType(), True),
        StructField("valor_descuento", DoubleType(), True),
        StructField("unidades", IntegerType(), True),
        StructField("total", DoubleType(), True),
        StructField("fecha_venta", DateType(), True)
    ])

    ventas_df = spark.createDataFrame(ventas_data, schema=ventas_schema)
    return ventas_df

# ==================================
# 🚀 Función principal
# ==================================
def main():
    fake = Faker("es_ES")
    # Clientes
    df_clientes = generar_clientes(fake, number_of_customers=10)
    # Productos
    df_productos = generar_productos()
    # Ventas
    df_ventas = generar_ventas(fake, df_clientes, df_productos, num_ventas=25)
    df_clientes.repartition(1).write.mode("overwrite").format("parquet").save(f"{volume_path}/medallion/clientes")
    df_productos.repartition(1).write.mode("overwrite").format("parquet").save(f"{volume_path}/medallion/productos")
    df_ventas.repartition(1).write.mode("overwrite").format("parquet").save(f"{volume_path}/medallion/ventas")

# Ejecutar
main()